In [3]:
#Importing all the dependencies

import pandas as pd
import itertools
import matplotlib.pyplot as plt
plt.style.use('default')
import seaborn as sns
import numpy as np
import re
import string
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.corpus import stopwords 
from textblob import Word, TextBlob
from wordcloud import WordCloud

from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
sia = SentimentIntensityAnalyzer()
import preprocessor as p
#from gensim.parsing.preprocessing import remove_stopwords

from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\reina\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [19]:
df = pd.read_csv('Datasets/fake_or_real_news_with_sentiment.csv')

In [20]:
df.head()

,Unnamed: 0,title,text,label,sentiment
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [21]:
df.shape

(6335, 5)

In [22]:
df['sentiment'].value_counts()

sentiment
1    5786
5     269
2     157
3      84
4      39
Name: count, dtype: int64

In [23]:
df=df.drop(['Unnamed: 0'], axis = 1)

In [24]:
df.head()

,title,text,label,sentiment
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [25]:
df['ID'] = range(1, len(df) + 1) #This creates a column titled ID, that starts at 1 and is the length of the DataFrame. 
first_column = df.pop('ID')
df.insert(0, 'Id', first_column) #This makes the newly created column the first in the dataframe.

In [26]:
df.head()

,Id,title,text,label,sentiment
0,1,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,1
1,2,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,1
2,3,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,1
3,4,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,1
4,5,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,1


In [27]:
#Removes null values
df.isnull().sum()

Id           0
title        0
text         0
label        0
sentiment    0
dtype: int64

In [28]:
df['text'] = df['text'].str.lower().str.replace('\\s+', ' ', regex=True).str.strip()

In [29]:
df.head()

,Id,title,text,label,sentiment
0,1,You Can Smell Hillary’s Fear,"daniel greenfield, a shillman journalism fello...",FAKE,1
1,2,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1
2,3,Kerry to go to Paris in gesture of sympathy,u.s. secretary of state john f. kerry said mon...,REAL,1
3,4,Bernie supporters on Twitter erupt in anger ag...,"— kaydee king (@kaydeeking) november 9, 2016 t...",FAKE,1
4,5,The Battle of New York: Why This Primary Matters,it's primary day in new york and front-runners...,REAL,1


In [30]:
import re
from nltk.corpus import stopwords

def prediction(row):
    text = row['text']
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    stemmed_review = [word for word in review if word not in stopwords.words('english')]
    return stemmed_review

# Apply the preprocessing function to the 'text' column
df['text'] = df.apply(prediction, axis=1)

# Display the first few rows of the DataFrame
df.head()


,Id,title,text,label,sentiment
0,1,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1
1,2,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1
2,3,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1
3,4,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1
4,5,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1


In [31]:
# tokens = nltk.word_tokenize('text')

In [32]:
# from nltk.corpus import stopwords

# stop_words=set(stopwords.words("english"))
# print(stop_words)

In [33]:
# tags = nltk.pos_tag(tokens)

Vader Model 

In [34]:
df.head()

,Id,title,text,label,sentiment
0,1,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1
1,2,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1
2,3,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1
3,4,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1
4,5,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1


In [35]:
from tqdm import tqdm
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize the VADER SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Get Polarity Scores
results = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['text']
    rowID = row['Id']
    # Join the list of tokens into a single string
    text_string = ' '.join(text)
    # Calculate polarity scores for the text
    results[rowID] = sia.polarity_scores(text_string)

# Create a DataFrame from the results
polarity_df = pd.DataFrame(results).T


100%|██████████| 6335/6335 [01:06<00:00, 95.54it/s] 


In [36]:
pd.DataFrame(results).T.describe()

,neg,neu,pos,compound
count,6335.000000,6335.000000,6335.000000,6335.000000
mean,0.125840,0.725253,0.143072,0.144351
std,0.076838,0.102670,0.066305,0.865486
min,0.000000,0.000000,0.000000,-1.000000
25%,0.073000,0.676000,0.103000,-0.931150
50%,0.117000,0.726000,0.141000,0.557400
75%,0.170000,0.777000,0.181000,0.981600
max,0.601000,1.000000,0.574000,1.000000


In [37]:
#Merge the polarity scores data frame with the main data frame
vader = polarity_df
vader = vader.reset_index().rename(columns={'index': 'Id'})
vader = vader.merge(df, how='left')
vader

,Id,neg,neu,pos,compound,title,text,label,sentiment
0,1,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1
1,2,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1
2,3,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1
3,4,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1
4,5,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1
...,...,...,...,...,...,...,...,...,...
6330,6331,0.045,0.898,0.057,0.6369,State Department says it can't find emails fro...,"[state, department, told, republican, national...",REAL,1
6331,6332,0.175,0.666,0.159,-0.9933,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,"[p, pbs, stand, plutocratic, pentagon, posted,...",FAKE,1
6332,6333,0.199,0.673,0.129,-0.9981,Anti-Trump Protesters Are Tools of the Oligarc...,"[anti, trump, protesters, tools, oligarchy, re...",FAKE,1
6333,6334,0.094,0.746,0.160,0.9920,"In Ethiopia, Obama seeks progress on peace, se...","[addis, ababa, ethiopia, president, obama, con...",REAL,1


In [38]:
#This is to compute whether it is Negative, Neutral or Positive
def GetAnalysis(score, tolerance=1e-6):
    if abs(score) < tolerance:
        return "Neutral"
    elif score < 0:
        return "Negative"
    else:
        return "Positive"

vader['VaderAnalysis']=vader['compound'].apply(GetAnalysis)
vader['VaderAnalysis'].head()

0    Negative
1    Positive
2    Positive
3    Positive
4    Positive
Name: VaderAnalysis, dtype: object

In [39]:
vader.head()

,Id,neg,neu,pos,compound,title,text,label,sentiment,VaderAnalysis
0,1,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1,Negative
1,2,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1,Positive
2,3,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1,Positive
3,4,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1,Positive
4,5,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1,Positive


In [40]:
vader.head()

,Id,neg,neu,pos,compound,title,text,label,sentiment,VaderAnalysis
0,1,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1,Negative
1,2,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1,Positive
2,3,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1,Positive
3,4,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1,Positive
4,5,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1,Positive


In [41]:
vader.drop('Id', axis=1, inplace=True)


In [42]:
vader.head()

,neg,neu,pos,compound,title,text,label,sentiment,VaderAnalysis
0,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1,Negative
1,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1,Positive
2,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1,Positive
3,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1,Positive
4,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1,Positive


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Define features (X) and target variable (y)
X = vader.drop(['label', 'title', 'text','VaderAnalysis'], axis=1)  # Excluding the 'label', 'title', and 'text' columns
y = vader['label']  # Target variable

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and fit the Linear Discriminant Analysis model
vm = LinearDiscriminantAnalysis().fit(X_train, y_train)

# Make predictions
predictions = vm.predict(X_test)



In [44]:
vader.head()

,neg,neu,pos,compound,title,text,label,sentiment,VaderAnalysis
0,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1,Negative
1,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1,Positive
2,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1,Positive
3,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1,Positive
4,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1,Positive


In [45]:
from sklearn.metrics import classification_report

# Print the classification report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

        FAKE       0.59      0.52      0.55       628
        REAL       0.58      0.64      0.61       639

    accuracy                           0.58      1267
   macro avg       0.58      0.58      0.58      1267
weighted avg       0.58      0.58      0.58      1267



In [46]:
vader.head()

,neg,neu,pos,compound,title,text,label,sentiment,VaderAnalysis
0,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,"[daniel, greenfield, shillman, journalism, fel...",FAKE,1,Negative
1,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,"[google, pinterest, digg, linkedin, reddit, st...",FAKE,1,Positive
2,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,"[u, secretary, state, john, f, kerry, said, mo...",REAL,1,Positive
3,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,"[kaydee, king, kaydeeking, november, lesson, t...",FAKE,1,Positive
4,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,"[primary, day, new, york, front, runners, hill...",REAL,1,Positive


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
import numpy as np

# Assuming 'vader' is your DataFrame containing the data

# Convert lists of text into single strings
vader['text'] = vader['text'].apply(lambda x: ' '.join(x))

# Initialize TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
text_features = tfidf_vectorizer.fit_transform(vader['text'])

# Combine text features with other features
other_features = vader[['neg', 'neu', 'pos']].values
combined_features = np.hstack((other_features, text_features.toarray()))

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_features, vader['sentiment'], test_size=0.2, random_state=42)

# Train the model
vm = LinearDiscriminantAnalysis().fit(X_train, y_train)
from sklearn.metrics import classification_report
# Make predictions
predictions = vm.predict(X_test)

# Print classification report
print(classification_report(y_test, predictions))




KeyboardInterrupt



Roberta Model

In [23]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [24]:
#Pretrained model by huggingface
roBERTa = f"cardiffnlp/twitter-roberta-base-sentiment" #PretrainedModel
tokenizer = AutoTokenizer.from_pretrained(roBERTa)
rb = AutoModelForSequenceClassification.from_pretrained(roBERTa)

# Initialize VADER
sia = SentimentIntensityAnalyzer()

In [25]:
#example
example='He does not like vegetables'
text_encoded=tokenizer(example,return_tensors='pt') #Eencodes the text into 1s and 0s for the model to understand
output = rb(**text_encoded)
scores = output[0][0].detach().numpy()#Roberta gives scores as tensors. this changes it to numpy. 
scores = softmax(scores) #Apply softmax to give the values between 0 and 1
scoresResult = {
    'roberta_neg' : scores[0],
    'roberta_neu' : scores[1],
    'roberta_pos' : scores[2]
}
print(scoresResult)

{'roberta_neg': 0.9027217, 'roberta_neu': 0.09143113, 'roberta_pos': 0.005847148}


In [21]:
#Create a function out if the code we ran on the example above to then apply to DataFrame after
def polarityRoBERTa(example):
    text_encoded=tokenizer(example,return_tensors='pt')
    output = rb(**text_encoded)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scoresResults = {
        'roberta_neg' : scores[0],
        'roberta_neu' : scores[1],
        'roberta_pos' : scores[2]
    }
    return scoresResults

In [19]:
def polarityRoBERTa_chunks(text, window_size=512, overlap=100):
    pos_scores = []
    neg_scores = []
    neu_scores = []

    chunks = [text[i:i+window_size] for i in range(0, len(text), window_size - overlap)]

    for chunk in chunks:
        result = polarityRoBERTa(chunk)
        pos_scores.append(result['roberta_pos'])
        neg_scores.append(result['roberta_neg'])
        neu_scores.append(result['roberta_neu'])

    # Calculate average scores for each sentiment category
    if pos_scores:
        avg_pos_score = sum(pos_scores) / len(pos_scores)
    else:
        avg_pos_score = 0
        
    if neg_scores:
        avg_neg_score = sum(neg_scores) / len(neg_scores)
    else:
        avg_neg_score = 0
    
    if neu_scores:
        avg_neu_score = sum(neu_scores) / len(neu_scores)
    else:
        avg_neu_score = 0

    # Create a dictionary of sentiment categories and their average scores
    avg_scores = {
        'roberta_pos': avg_pos_score,
        'roberta_neg': avg_neg_score,
        'roberta_neu': avg_neu_score
    }

    # Sort the dictionary based on scores in descending order
    sorted_avg_scores = dict(sorted(avg_scores.items(), key=lambda item: item[1], reverse=True))

    # Return the sorted average scores dictionary
    return sorted_avg_scores


In [20]:
polarityRoBERTa_chunks("Hello i am sad")

{'roberta_neg': 0.9128637313842773,
 'roberta_neu': 0.07640038430690765,
 'roberta_pos': 0.010735953226685524}

In [ ]:
polarityRoBERTa("Hello i am sad")

{'roberta_neg': 0.91286373,
 'roberta_neu': 0.076400384,
 'roberta_pos': 0.010735953}

In [ ]:
vader['ID'] = range(1, len(vader) + 1) #This creates a column titled ID, that starts at 1 and is the length of the DataFrame. 
first_column = vader.pop('ID')
vader.insert(0, 'ID', first_column) #This makes the newly created column the first in the dataframe.

In [ ]:
vader.head()

,ID,neg,neu,pos,compound,title,text,label,sentiment,VaderAnalysis
0,1,0.262,0.624,0.114,-0.9991,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative
1,2,0.114,0.729,0.157,0.9063,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive
2,3,0.104,0.758,0.138,0.6597,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive
3,4,0.124,0.747,0.129,0.0680,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive
4,5,0.107,0.665,0.229,0.9812,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive


In [ ]:
#THIS IS THE CODE THAT TAKES 2 HOURS TO RUN :_) IT CHANGES FROM THE ABOVE DF TO THE ONE BELOW!! 


# batch_size = 100  # Set the batch size

# results = {}
# num_batches = (len(vader) + batch_size - 1) // batch_size  # Calculate the number of batches

# # Iterate over batches
# for batch_idx in tqdm(range(num_batches)):
#     batch_start = batch_idx * batch_size
#     batch_end = min((batch_idx + 1) * batch_size, len(vader))
#     batch_data = vader.iloc[batch_start:batch_end]

#     # Process each row in the batch
#     for i, row in batch_data.iterrows():
#         texts = row['text']
#         rowID = row['ID']
        
#         #VADER sentiment analysis
#         vader_results = sia.polarity_scores(texts)
#         vader_result_rename = {}
        
#         #Display vader results
#         for key, value in vader_results.items():
#             vader_result_rename[f"vader_{key}"] = value
        
#         #RoBERTa sentiment analysis
#         roberta_results = polarityRoBERTa_chunks(texts)
        
#         # Combine VADER and RoBERTa results
#         both = {**vader_result_rename, **roberta_results} 
#         results[rowID] = both

# # Create DataFrame for combined results
# resultsDF = pd.DataFrame(results).T.reset_index().rename(columns={'index': 'ID'})

# # Merge with original DataFrame
# resultsDF = resultsDF.merge(vader, how='left')

# # Drop unwanted columns
# resultsDF = resultsDF.drop(['neg', 'pos', 'neu', 'compound'], axis=1)


  0%|          | 0/64 [00:00<?, ?it/s]

100%|██████████| 64/64 [2:10:03<00:00, 121.93s/it]  


In [ ]:
resultsDF = vader


,ID,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,title,text,label,sentiment,VaderAnalysis
0,1,0.262,0.624,0.114,-0.9991,0.575292,0.402147,0.022560,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative
1,2,0.114,0.729,0.157,0.9063,0.603442,0.349584,0.046974,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive
2,3,0.104,0.758,0.138,0.6597,0.231290,0.697491,0.071219,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive
3,4,0.124,0.747,0.129,0.0680,0.324970,0.565880,0.109150,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive
4,5,0.107,0.665,0.229,0.9812,0.100435,0.705917,0.193647,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive


In [ ]:
#Use lambda function to label tweet as positive, negative or neutral
resultsDF['Rob_Analysis'] = resultsDF.apply(lambda x:'Negative' if x['roberta_neg'] >
                     x['roberta_neu'] and x['roberta_neu']
                     > x['roberta_pos'] else ('Neutral' if x['roberta_neu']>x['roberta_pos'] else 'Positive'),axis=1)
resultsDF['Rob_Analysis'].head()

0    Negative
1    Negative
2     Neutral
3     Neutral
4     Neutral
Name: Rob_Analysis, dtype: object

RoBERTa does not compute the compound score so here we have to compare the 3 probabilities and the Highest Score becomes the label of the text.

In [ ]:
resultsDF.head()

,ID,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,title,text,label,sentiment,VaderAnalysis,Rob_Analysis
0,1,0.262,0.624,0.114,-0.9991,0.575292,0.402147,0.022560,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative,Negative
1,2,0.114,0.729,0.157,0.9063,0.603442,0.349584,0.046974,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive,Negative
2,3,0.104,0.758,0.138,0.6597,0.231290,0.697491,0.071219,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive,Neutral
3,4,0.124,0.747,0.129,0.0680,0.324970,0.565880,0.109150,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive,Neutral
4,5,0.107,0.665,0.229,0.9812,0.100435,0.705917,0.193647,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive,Neutral


In [ ]:
sortedDF.head()

,ID,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,title,text,label,sentiment,VaderAnalysis,Rob_Analysis
0,1,0.262,0.624,0.114,-0.9991,0.575292,0.402147,0.022560,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative,Negative
1,2,0.114,0.729,0.157,0.9063,0.603442,0.349584,0.046974,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive,Negative
2,3,0.104,0.758,0.138,0.6597,0.231290,0.697491,0.071219,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive,Neutral
3,4,0.124,0.747,0.129,0.0680,0.324970,0.565880,0.109150,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive,Neutral
4,5,0.107,0.665,0.229,0.9812,0.100435,0.705917,0.193647,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive,Neutral


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

def combined_sentiment_analysis(row, weight_roberta=0.6, weight_vader=0.4):
    # Getting sentiment from RoBERTa
    roberta_score = 1 if row['roberta_pos'] > row['roberta_neg'] else (-1 if row['roberta_neg'] > row['roberta_pos'] else 0)

    # Get sentiment from VADER
    vader_score = row['vader_compound']

    # Calculate combined score
    combined_score = (weight_roberta * roberta_score) + (weight_vader * vader_score)

    # Return combined sentiment
    return "POSITIVE" if combined_score > 0 else ("NEGATIVE" if combined_score < 0 else "NEUTRAL")

# Assuming you have a dataframe called sortedDF
sortedDF['combined_sentiment'] = sortedDF.apply(combined_sentiment_analysis, axis=1)

# Print the combined sentiment column
print("Combined Sentiment:")
print(sortedDF['combined_sentiment'])


Combined Sentiment:
0       NEGATIVE
1       NEGATIVE
2       NEGATIVE
3       NEGATIVE
4       POSITIVE
          ...   
6330    NEGATIVE
6331    NEGATIVE
6332    NEGATIVE
6333    NEGATIVE
6334    NEGATIVE
Name: combined_sentiment, Length: 6335, dtype: object


In [ ]:
sortedDF.head()

,ID,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,title,text,label,sentiment,VaderAnalysis,Rob_Analysis,combined_sentiment
0,1,0.262,0.624,0.114,-0.9991,0.575292,0.402147,0.022560,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative,Negative,NEGATIVE
1,2,0.114,0.729,0.157,0.9063,0.603442,0.349584,0.046974,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive,Negative,NEGATIVE
2,3,0.104,0.758,0.138,0.6597,0.231290,0.697491,0.071219,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive,Neutral,NEGATIVE
3,4,0.124,0.747,0.129,0.0680,0.324970,0.565880,0.109150,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive,Neutral,NEGATIVE
4,5,0.107,0.665,0.229,0.9812,0.100435,0.705917,0.193647,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive,Neutral,POSITIVE


THE CODE BELOW NEEDS TO GET CHANGED. WE CAN NOW IMPORT THE DATA FRAME TOO!

In [6]:
df = pd.read_csv('sortedDataFrame.csv')



In [8]:
df.isnull().sum()

Unnamed: 0             0
ID                     0
vader_neg              0
vader_neu              0
vader_pos              0
vader_compound         0
roberta_neg            0
roberta_neu            0
roberta_pos            0
title                  0
text                  37
label                  0
sentiment              0
VaderAnalysis          0
Rob_Analysis           0
combined_sentiment     0
dtype: int64

In [9]:
# Find indices where 'text' column is null
null_text_indices = df[df['text'].isnull()].index

# Delete rows with null 'text'
df.drop(null_text_indices, inplace=True)

In [10]:
df.isnull().sum()

Unnamed: 0            0
ID                    0
vader_neg             0
vader_neu             0
vader_pos             0
vader_compound        0
roberta_neg           0
roberta_neu           0
roberta_pos           0
title                 0
text                  0
label                 0
sentiment             0
VaderAnalysis         0
Rob_Analysis          0
combined_sentiment    0
dtype: int64

In [11]:
df.head()

,Unnamed: 0,ID,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,title,text,label,sentiment,VaderAnalysis,Rob_Analysis,combined_sentiment
0,0,1,0.262,0.624,0.114,-0.9991,0.575292,0.402147,0.022560,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative,Negative,NEGATIVE
1,1,2,0.114,0.729,0.157,0.9063,0.603442,0.349584,0.046974,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive,Negative,NEGATIVE
2,2,3,0.104,0.758,0.138,0.6597,0.231290,0.697491,0.071219,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive,Neutral,NEGATIVE
3,3,4,0.124,0.747,0.129,0.0680,0.324970,0.565880,0.109150,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive,Neutral,NEGATIVE
4,4,5,0.107,0.665,0.229,0.9812,0.100435,0.705917,0.193647,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive,Neutral,POSITIVE


In [12]:
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,ID,vader_neg,vader_neu,vader_pos,vader_compound,roberta_neg,roberta_neu,roberta_pos,title,text,label,sentiment,VaderAnalysis,Rob_Analysis,combined_sentiment
0,1,0.262,0.624,0.114,-0.9991,0.575292,0.402147,0.022560,You Can Smell Hillary’s Fear,daniel greenfield shillman journalism fellow f...,FAKE,1,Negative,Negative,NEGATIVE
1,2,0.114,0.729,0.157,0.9063,0.603442,0.349584,0.046974,Watch The Exact Moment Paul Ryan Committed Pol...,google pinterest digg linkedin reddit stumbleu...,FAKE,1,Positive,Negative,NEGATIVE
2,3,0.104,0.758,0.138,0.6597,0.231290,0.697491,0.071219,Kerry to go to Paris in gesture of sympathy,u secretary state john f kerry said monday sto...,REAL,1,Positive,Neutral,NEGATIVE
3,4,0.124,0.747,0.129,0.0680,0.324970,0.565880,0.109150,Bernie supporters on Twitter erupt in anger ag...,kaydee king kaydeeking november lesson tonight...,FAKE,1,Positive,Neutral,NEGATIVE
4,5,0.107,0.665,0.229,0.9812,0.100435,0.705917,0.193647,The Battle of New York: Why This Primary Matters,primary day new york front runners hillary cli...,REAL,1,Positive,Neutral,POSITIVE


In [9]:
import pandas as pd

# Assuming df is your DataFrame
column_name = 'roberta_pos'  # Replace 'your_column_name' with the actual column name
total_rows_in_column = df[column_name].count()

print("Total number of rows in the column:", total_rows_in_column)


Total number of rows in the column: 6298


In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Initialize TfidfVectorizer for text features
tfidf_vectorizer_text = TfidfVectorizer()
tfidf_vectorizer_combined_sentiment = TfidfVectorizer()

# Fit and transform the 'text' feature
X_text_features_text = tfidf_vectorizer_text.fit_transform(df['text'])

# Fit and transform the 'combined_sentiment' feature
X_text_features_combined_sentiment = tfidf_vectorizer_combined_sentiment.fit_transform(df['combined_sentiment'])

# Combine text features into one feature matrix
X = pd.concat([pd.DataFrame(X_text_features_text.toarray()), 
               pd.DataFrame(X_text_features_combined_sentiment.toarray())], axis=1)

# Target variable
y = df['label']

# Split the df into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Selection and Training
rf_model = RandomForestClassifier()  # Example, you can try different models
rf_model.fit(X_train, y_train)

# Model Evaluation
y_pred_rf = rf_model.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Random Forest Accuracy:", accuracy_rf)


Random Forest Accuracy: 0.9015873015873016


In [12]:
from sklearn.metrics import classification_report
# Print classification report
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

        FAKE       0.90      0.90      0.90       634
        REAL       0.90      0.90      0.90       626

    accuracy                           0.90      1260
   macro avg       0.90      0.90      0.90      1260
weighted avg       0.90      0.90      0.90      1260



We now need to test a sample text?
prediction() process the text.
sia.polarity_scores(text_string) vader sentiment scores with pos,neg,neu
getAnalysis() returns pos or neg or neu based on vader sentiment scores
polarity_RobertaChunks() returns pos,neg,neu with scores

resultsDF['Rob_Analysis'] = resultsDF.apply(lambda x:'Negative' if x['roberta_neg'] >
                     x['roberta_neu'] and x['roberta_neu']
                     > x['roberta_pos'] else ('Neutral' if x['roberta_neu']>x['roberta_pos'] else 'Positive'),axis=1)

for roberta scores tells us whether it's overall pos,neg,neu

combined_sentiment_analysis() tells us overall sentiment

# Then we use this!

prediction_rf = rf_model.predict(new_X)

# Print the prediction
if prediction_rf[0] == 1:
    print("The Random Forest model predicts that the text is real.")
else:
    print("The Random Forest model predicts that the text is fake.")
# Check whether it's real or fake!!

In [28]:
#Testing random stuff
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
# Open the file with UTF-8 encoding
file_path = "test.txt"  # Provide the correct file path if necessary
with open(file_path, "r", encoding="utf-8") as file:
    test_text = file.read()


def prediction(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    stemmed_review = [word for word in review if word not in stopwords.words('english')]
    text_string = ' '.join(stemmed_review)
    return text_string
def GetAnalysis(score):
    if -0.2 < score < 0.2:
        return "Neutral"
    elif score < 0:
        return "Negative"
    else:
        return "Positive"
def robertaAnalysis(neg, neu, pos):
    if neg > neu and neg > pos:
        return "Negative"
    elif pos > neu and pos > neg:
        return "Positive"
    else:
        return "Neutral"
def combined_sentiment_analysis(roberta,vader, weight_roberta=0.6, weight_vader=0.4):


    roberta_score = 1 if roberta == "Positive" else (-1 if roberta == "Negative" else 0)

    vader_score = vader
    # Calculate combined score
    combined_score = (weight_roberta * roberta_score) + (weight_vader * vader_score)

    # Return combined sentiment
    return "POSITIVE" if combined_score > 0 else ("NEGATIVE" if combined_score < 0 else "NEUTRAL")





test_text = prediction(test_text)


vaderScores = sia.polarity_scores(test_text)

vaderFinal = GetAnalysis(vaderScores['compound'])
print(vaderScores)

robertaScores = polarityRoBERTa_chunks(test_text)
robertaFinal = robertaAnalysis(robertaScores['roberta_neg'],robertaScores['roberta_neu'],robertaScores['roberta_pos'])
print(robertaScores)

print("***********")
combined_sentiment = combined_sentiment_analysis(robertaFinal,vaderScores['compound'])
print(combined_sentiment)

test_text_features_text = tfidf_vectorizer_text.transform([test_text])
test_text_features_combined_sentiment = tfidf_vectorizer_combined_sentiment.transform([test_text])
# Combine the text features into one feature matrix
test_features = pd.concat([pd.DataFrame(test_text_features_text.toarray()), 
                           pd.DataFrame(test_text_features_combined_sentiment.toarray())], axis=1)

# Predict the label using the trained RandomForestClassifier
predicted_label = rf_model.predict(test_features)

# Print or return the predicted label
print("Predicted Label:", predicted_label)


{'neg': 0.079, 'neu': 0.672, 'pos': 0.249, 'compound': 0.9899}
{'roberta_neu': 0.7344753742218018, 'roberta_neg': 0.1638067662715912, 'roberta_pos': 0.1017178826034069}
***********
POSITIVE
Predicted Label: ['FAKE']


In [15]:
path = "test.txt"
with open(path, 'r', encoding = 'utf-8') as file:
    sample_text = file.read()
def prediction(text):
    review = re.sub('[^a-zA-Z]', ' ', text)
    review = review.lower().split()
    stemmed_review = [word for word in review if word not in stopwords.words('english')]
    text_string = ' '.join(stemmed_review)
    return text_string

text = prediction(sample_text)

tfidf_sampletext = tfidf_vectorizer_text.transform([text])
tfidf_sentiment = tfidf_vectorizer_combined_sentiment.transform([text])

test_features = pd.concat([pd.DataFrame(tfidf_sampletext.toarray()), 
                           pd.DataFrame(tfidf_sentiment.toarray())], axis=1)
samplePred = rf_model.predict(test_features)

print("Predicted Label: ", samplePred)

Predicted Label:  ['FAKE']
